In [ ]:
# necessary for DenodoConnection
#%pip install psycopg2-binary
#%pip install shap
#%pip install scikit-optimize

# generate train and validation sets
# during training, make K models via k-folds cross validation
# during evaluation, evaluate on validation set (completely left out).

# stages 1,3, 4 for Corn Brazil Safrinha, advancements made on hybrids
# do we combine stages to make 1 model?

In [2]:
# import packages
import os
import sys

import pandas as pd, numpy as np
import argparse

# need to add dme_sagemaker to path to load in libraries
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.performance_lib import predictive_advancement_lib
from libs.performance_lib import performance_helper

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
DKU_DST_ap_data_sector = 'CORN_BRAZIL_SAFRINHA'
DKU_DST_analysis_type = 'SingleExp'
input_years = ['2018','2019','2020','2021','2022']
bucket = 'us.com.syngenta.ap.nonprod'

# convert each entry in input years to an integer since they come in as str.
if isinstance(input_years, list):
    input_years_as_int = [int(val) for val in input_years]
else:
    input_years_as_int = [int(input_years)]

In [4]:
# when testing, we read directly from S3, bucket is provided as a separate input to function, don't include in input args. 
input_args = [
    '--s3_input_pred_adv_data_folder', 'uat/dme/performance/compute_pred_adv_data_collected/data/'
]

parser = argparse.ArgumentParser(description='app inputs and outputs')
parser.add_argument('--s3_input_pred_adv_data_folder', type=str,
                    help='s3 input pred_adv_data folder', required=True)
args = parser.parse_args(input_args)

In [5]:
potential_fnames = predictive_advancement_lib.load_potential_fnames(prefix='new_')

df_input_piv = predictive_advancement_lib.load_and_preprocess_all_inputs_ml(
    args,
    DKU_DST_ap_data_sector,
    DKU_DST_analysis_type,
    potential_fnames,
    years_to_load=input_years,
    read_from_s3=1,
    bucket=bucket
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


(12805, 42)
(33296, 9)
(16879, 42)
(61321, 9)
(14053, 41)
(59031, 9)
(16062, 42)
(65957, 9)
(28801, 47)
(29724, 50)
(123, 4)
(77502, 9)


In [6]:
df_input_piv.shape

(88600, 100)

In [7]:
df_input_piv[df_input_piv['material_type_simple'] == 'entry'][['entry_identifier','current_stage','analysis_year']].groupby(by=['current_stage','analysis_year']).count()

entry_identifier
current_stage analysis_year                  
1.0           2018                       5824
              2019                       8955
              2020                       7445
              2021                       8545
              2022                      15838
3.0           2018                       2793
              2019                       5570
              2020                       5531
              2021                       5602
              2022                       6621
4.0           2018                       1511
              2019                       1944
              2020                        676
              2021                       1356
              2022                       5821
5.0           2018                        174
              2019                        150
              2020                         85
              2021                        324
              2022                        296
6.0           2018                         11
              2019                         38
              2020                        233
              2021                         44
              2022                         43
7.0           2018                         52
              2019                        141
              2020                         72
              2021                         99
              2022                         68

In [8]:
for col in df_input_piv.columns:
    print(col, df_input_piv[col].count())

ap_data_sector 88600
analysis_year 88600
entry_identifier 88600
decision_group_rm 88600
material_type_simple 88600
chkfl_ERHTN 55661
chkfl_GMSTP 87783
chkfl_GWTPN 87704
chkfl_HAVPN 3326
chkfl_LRTLR 79203
chkfl_PLHTN 48317
chkfl_STD_N 14487
chkfl_STKLR 77274
chkfl_TWSMN 29857
chkfl_YGHMN 87710
chkfl_YGMRN 87710
chkfl_YGSMN 87710
cpifl 148
result_ERHTN 55661
result_GMSTP 87783
result_GWTPN 87704
result_HAVPN 3326
result_LRTLR 79203
result_PLHTN 48317
result_STD_N 14487
result_STKLR 77274
result_TWSMN 29857
result_YGHMN 87710
result_YGMRN 87710
result_YGSMN 87710
result_diff_ERHTN 55661
result_diff_GMSTP 87783
result_diff_GWTPN 87704
result_diff_HAVPN 3326
result_diff_LRTLR 79203
result_diff_PLHTN 48317
result_diff_STD_N 14487
result_diff_STKLR 77274
result_diff_TWSMN 29857
result_diff_YGHMN 87710
result_diff_YGMRN 87710
result_diff_YGSMN 87710
current_stage 85862
next_stage 85862
prev_stage 6416
third_stage 85862
was_adv 85862
was_adv_next 85862
chkfl_FAL1R 55
chkfl_FAWLR 55
result_FAL1R

In [9]:
df_input_piv[['was_adv','current_stage','analysis_year']].groupby(by=['current_stage','analysis_year']).sum()

was_adv
current_stage analysis_year         
1.0           2018              69.0
              2019              53.0
              2020              19.0
              2021             733.0
              2022            1053.0
3.0           2018              58.0
              2019              76.0
              2020             257.0
              2021             569.0
              2022             160.0
4.0           2018              36.0
              2019              92.0
              2020              17.0
              2021             124.0
              2022             223.0
5.0           2018               2.0
              2019              14.0
              2020              13.0
              2021              12.0
              2022              35.0
6.0           2018               0.0
              2019               0.0
              2020               0.0
              2021               0.0
              2022               0.0
7.0           2018               0.0
              2019               0.0
              2020               0.0
              2021               0.0
              2022               0.0

In [10]:
#### FILTER TO DESIRED DATA for training
# for corn brazil safrinha: stage 1, 3 and 4.
df_piv = df_input_piv[(df_input_piv['material_type_simple'] == 'entry') &
                      ((df_input_piv['current_stage']==1) | (df_input_piv['current_stage']==3) | (df_input_piv['current_stage'] == 4))]


if 'decision_group' not in df_piv.columns:
    df_piv['decision_group'] = 'na'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
# get columns used for training/evaluation + meta columns
stratify_cols = ['analysis_year','current_stage']

# split into training and validation sets. Use stratified sampling.
df_tr_all, df_val_all = predictive_advancement_lib.stratified_train_test_split(df_piv, stratify_cols)

# provide label for k-fold training in training set. Don't need to split data, just label it once. Do this in a stratified manner as well 
df_tr_all = predictive_advancement_lib.stratified_kfolds(df_tr_all, stratify_cols, n_folds=5)

/root/dme_sagemaker/dme_sagemaker/libs/performance_lib/predictive_advancement_lib.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_in_val['kfold_label'] = kfold_label


In [14]:
# save training and validation sets. <- this will change in the pipeline to some other saving methods...
s3_path=os.path.join('uat/dme/performance/reformatting_performance_pipeline_temp_out/data',DKU_DST_ap_data_sector)
local_fpath = '/root/dme_sagemaker/dme_sagemaker/performance_pipeline/preprocess_train_recipes/'
performance_helper.write_to_s3(
    obj=df_tr_all,
    fname='adv_model_training_data.csv', 
    local_fpath=local_fpath,
    s3_path=s3_path
)

performance_helper.write_to_s3(
    df_val_all,
    fname='adv_model_validation_data.csv', 
    local_fpath=local_fpath,
    s3_path=s3_path
)
### end preprocessing_training.py